In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mafat-mini/mini_training_set.db
/kaggle/input/mafat-mini/training_set.db


In [2]:
import pandas as pd
import sqlite3
import glob

import numpy as np
import xgboost
import json
from sklearn.metrics import roc_auc_score
from sklearn.feature_selection import RFE
from sklearn.model_selection import StratifiedShuffleSplit
print(pd.__version__)

2.2.3


# functions

In [3]:
def target_extract(conn):
    """
    This function return The Target of each Device_ID.

    Parameters
    ----------
    conn: connection
        A connection object to the database.

    Returns
    -------
    dataframe
        A dataframe of the labels.
    """
    cur = conn.cursor()
    sql = '''SELECT DISTINCT Device_ID, Target
             FROM data;
             '''
    cur.execute(sql)
    target = pd.DataFrame(cur.fetchall(), columns = ['Device_ID', 'Target'])
    target['Target'] = target['Target'].astype('int')
    return target

In [4]:
def rename_and_16_convert(dataset,prefix):
    """
    Processing data to reduce memory and creating unique columns names for features.

    Parameters
    ----------
    dataset: dataframe
        A dataframe columns includes Device_ID and features to process.
    prefix : str
        A string to concatenate to the feature names.

    Returns
    -------
    dataframe
        The processed data inclued Device_ID column.
    """
    col_dataset = list(dataset.drop(['Device_ID'], axis=1).columns)
    df_device = dataset['Device_ID']

    dataset.loc[:,col_dataset] *=1000
    dataset = dataset.loc[:,col_dataset].astype('float16')
    dataset = pd.concat([dataset, df_device], axis=1)

    new_columns_name = {n: f'{prefix}_{n}'for n in col_dataset}

    dataset.rename(columns=new_columns_name, inplace=True)
    return dataset


    

def relative_domain(conn, device_list):
    """
    Feature engeinering: For each Device_ID calculate the proportions of all the domain_Name he entered.

    Parameters
    ----------
    conn: connection
        A connection object to the database.
    device_list : list
        A list of Device_IDs to calculate their proportions.

    Returns
    -------
    dataframe
        A dataframe with the proportions for each Device_IDs and Domain_Name.
    """
    list_device_str = ', '.join(map(str, device_list))
    cur = conn.cursor()
    sql = f'''SELECT DISTINCT
                Device_ID,
                Domain_Name,
                Domain_Name_count*1.0 / SUM(Domain_Name_count) OVER (PARTITION BY Device_ID) AS relative_domain
            FROM (
              SELECT Device_ID, Domain_Name, COUNT(*) as Domain_Name_count
            		FROM data
            		WHERE Device_ID IN (''' +list_device_str+''')
                    GROUP BY Device_ID, Domain_Name
            		) subquery;'''
    cur.execute(sql)
    df = pd.DataFrame(cur.fetchall(), columns = ['Device_ID','Domain_Name', 'relative_domain'])

    df = df.pivot_table(index='Device_ID', columns='Domain_Name', values='relative_domain', fill_value=0)
    df['Device_ID'] = df.index
    df.reset_index(inplace=True, drop=True)
    return df

def cls_proportion(conn, device_list):
    """
    Feature engeinering: For each Device_ID calculate the proportions of all the domain_cls he entered.

    Parameters
    ----------
    conn: connection
        A connection object to the database.
    device_list : list
        A list of Device_IDs to calculate their proportions.

    Returns
    -------
    dataframe
        A dataframe with the proportions for each Device_IDs and Domain_cls.
    """
    list_device_str = ', '.join(map(str, device_list))
    cur = conn.cursor()
    sql = '''SELECT
                    Device_ID,
                    Domain_cls,
                    CAST(count_cls AS REAL) / SUM(count_cls) OVER (PARTITION BY Device_ID) AS proportion
                    FROM
                    (SELECT Device_ID, Domain_cls , COUNT(*) AS count_cls
                    FROM (
                        SELECT Device_ID, Domain_cls1 AS Domain_cls FROM data WHERE (Domain_cls1 != 0 AND Device_ID IN (''' +list_device_str+'''))
                        UNION ALL
                        SELECT Device_ID, Domain_cls2 AS Domain_cls FROM data WHERE (Domain_cls2 != 0 AND Device_ID IN (''' +list_device_str+'''))
                        UNION ALL
                        SELECT Device_ID, Domain_cls3 AS Domain_cls FROM data WHERE (Domain_cls3 != 0 AND Device_ID IN (''' +list_device_str+'''))
                        UNION ALL
                        SELECT Device_ID, Domain_cls4 AS Domain_cls FROM data WHERE (Domain_cls4 != 0 AND Device_ID IN (''' +list_device_str+'''))
                    ) AS combined
                    WHERE Domain_cls!=0
                    GROUP BY Device_ID, Domain_cls
                    ORDER BY Device_ID, Domain_cls)
                    subquery;'''

    cur.execute(sql)
    df = pd.DataFrame(cur.fetchall(), columns = ['Device_ID','Domain_cls', 'proportion'])

    df = df.pivot_table(index='Device_ID', columns='Domain_cls', values='proportion', fill_value=0)
    df['Device_ID'] = df.index
    df.reset_index(inplace=True, drop=True)
    return df

def avg_relative_entrances_device_id(conn, hours_duration, device_list):
    """
    Feature engeinering: for each Device_ID calculation of the proportional hits according to the day's parts.
    Calculation of proportional hits: For each Device_ID, sum up the proportional hits for each day's part (calculated each day) and divide them by the number of days (all days of internet usage -queries).

    Parameters
    ----------
    conn: connection
        A connection object to the database.
    hours_duration : int
        The interval duration of each day's parts in hours (Day division to 24/'hours_duration' parts).
    device_list : list
        A list of Device_IDs to calculate their proportions.

    Returns
    -------
    dataframe
        A dataframe with the proportional hits for each Device_ID and time_range.
    """
    df_sum_relative = sum_relative_entrances_timerange(conn, hours_duration, device_list)

    all_desired_combinations = list(pd.MultiIndex.from_product([df_sum_relative['Device_ID'].unique(), range(int(24/hours_duration))], names=['Device_ID', 'time_range']))
    diff_to_add = set(all_desired_combinations).difference(set(df_sum_relative.apply(lambda row: (row['Device_ID'], row['time_range']), axis=1).to_list()))
    diff_to_add = [x +(0,) for x in diff_to_add]
    diff_to_add_df = pd.DataFrame(diff_to_add, columns = list(df_sum_relative.columns))
    df_sum_relative = pd.concat([df_sum_relative,diff_to_add_df], axis=0)
    df_sum_relative.reset_index(drop = True, inplace = True)

    df_days_count_train = count_day_device_id(conn, device_list)

    df = pd.merge(df_sum_relative, df_days_count_train, how ='left', on ='Device_ID')
    df['relative_part'] = df['sum_relative_part']/df['day_num']
    df.drop(['day_num','sum_relative_part'],axis=1, inplace = True)

    df = df.pivot_table(index='Device_ID', columns='time_range', values='relative_part', fill_value=0)
    df['Device_ID'] = df.index
    df.reset_index(inplace=True, drop=True)

    return df

def sum_relative_entrances_timerange(conn, hours_duration, device_list):
    """
    For each Device_ID, sum the proportional hits in each day according to the day's parts.

    Parameters
    ----------
    conn: connection
        A connection object to the database.
    hours_duration : int
        The interval duration of each day's parts in hours (Day division to 24/'hours_duration' parts).
    device_list:  list
        A list of Device_IDs to calculate their proportions.

    Returns
    -------
    dataframe
        A dataframe contains Device_ID, part of the day, and the sum of the proportional hits.
    """
    list_device_str = ', '.join(map(str, device_list))
    cur = conn.cursor()
    sql = f'''SELECT DISTINCT
                        Device_ID,
                        time_range,
                        SUM (relative_part) OVER (PARTITION BY Device_ID,time_range) AS sum_relative_part
                    FROM(

                            SELECT distinct
                                            Device_ID,
                                            date,
                                            time_range,
                                            CAST(COUNT(*) OVER (PARTITION BY Device_ID,date,time_range) AS REAL) / COUNT(*) OVER (PARTITION BY Device_ID,date) AS relative_part
                                        FROM
                                                    (SELECT
                                                            Device_ID,
                                                            Datetime,
                                                            strftime('%Y-%m-%d', Datetime) AS date,
                                                            (CAST(strftime('%H', Datetime) AS INTEGER) / {hours_duration}) AS time_range
                                                        FROM
                                                            data
                                                        WHERE
                                                            Device_ID IN (''' +list_device_str+''')
                                                    ) subquery
                        ) subquery
                        ;'''
    cur.execute(sql)
    df = pd.DataFrame(cur.fetchall(), columns = ['Device_ID','time_range', 'sum_relative_part'])
    return df

def count_day_device_id(conn, device_list):
    """
    This function counts the days with internet usage(queries) of each Device_ID.

    Parameters
    ----------
    conn: connection
        A connection object to the database.
    device_list:  list
        A list of Device_IDs to calculate their proportions.

    Returns
    -------
    dataframe
        A dataframe contains Device_ID and total days.
    """
    list_device_str = ', '.join(map(str, device_list))
    cur = conn.cursor()
    sql = f'''
                SELECT
                    Device_ID,
                    COUNT(DISTINCT strftime('%Y-%m-%d', Datetime)) AS day_num
                FROM
                    data
                WHERE
                    Device_ID IN (''' +list_device_str+''')
                GROUP BY
                    Device_ID
                ;'''
    cur.execute(sql)
    df = pd.DataFrame(cur.fetchall(), columns = ['Device_ID', 'day_num'])
    return df

def corresponding_columns_training_set(df_train_col_list, df):
    """
    This function checks the gaps between the features received as arguments and the data's columns. And changes the columns' data to be the same as those received as arguments.

    Parameters
    ----------
    df_train_col_list: list
        List of features from the training set
    df:  dataframe
        A dataset whose columns will be changed according to df_train_col_list.

    Returns
    -------
    dataframe
        A dataframe with columns compatible with those of the training set.
    """
    del_col = set(list(df.columns)) - set(df_train_col_list)
    df.drop(columns = del_col, inplace = True)
    diff_col = set(df_train_col_list)-set(list(df.columns))
    add_to_test = pd.DataFrame(0, index=np.arange(len(df)), columns=list(diff_col)).astype('float16')
    df = pd.concat([df, add_to_test], axis=1)
    return df




In [5]:
def basic_stats(conn, device_list):
 
    list_device_str = ', '.join(map(str, device_list))
    cur = conn.cursor()

    sql = f'''
    SELECT Device_ID,total_urls,unique_urls,total_dates,unique_domains,unique_cls1,
    diff_mean,diff_std,diff_max,min_nonzero_cls,max_nonzero_cls,mean_nonzero_cls,std_nonzero_cls,

 sum_equal_0,sum_equal_1,sum_equal_2,sum_equal_3,

 (CAST (sum_equal_0 as REAL)/total_urls) as sum_equal_0_perc,
 (CAST (sum_equal_1 as REAL)/total_urls) as sum_equal_1_perc,
 (CAST (sum_equal_2 as REAL)/total_urls) as sum_equal_2_perc,
 (CAST (sum_equal_3 as REAL)/total_urls) as sum_equal_3_perc

      FROM ( SELECT 
        d.Device_ID,
        COUNT(d.URL) AS total_urls,
        
        COUNT(DISTINCT d.URL) AS unique_urls,
        COUNT(DISTINCT d.Datetime) AS total_dates,
        
        COUNT(DISTINCT d.Domain_Name) AS unique_domains,
        COUNT(DISTINCT d.Domain_cls1) AS unique_cls1,
        
        AVG(diff) AS diff_mean,
        SQRT(AVG(diff * diff) - (AVG(diff) * AVG(diff))) AS diff_std,
        MAX(diff) AS diff_max,

        MIN(nonzero_csl) as min_nonzero_cls,
        MAX(nonzero_csl) as max_nonzero_cls,
        AVG(nonzero_csl) as mean_nonzero_cls,
        SQRT(AVG(nonzero_csl * nonzero_csl) - (AVG(nonzero_csl) * AVG(nonzero_csl))) as std_nonzero_cls,

        SUM( CASE WHEN nonzero_csl =0 THEN 1 ELSE 0 END)  as sum_equal_0,
        SUM( CASE WHEN nonzero_csl =1 THEN 1 ELSE 0 END)  as sum_equal_1,
        SUM( CASE WHEN nonzero_csl =2 THEN 1 ELSE 0 END)  as sum_equal_2,
        SUM( CASE WHEN nonzero_csl =3 THEN 1 ELSE 0 END)  as sum_equal_3
        
        FROM (
        SELECT 
            Device_ID,
            Datetime,
            URL,
            Domain_Name,
            Domain_cls1,
            (CASE WHEN Domain_cls1 != 0 THEN 1 ELSE 0 END +
            CASE WHEN Domain_cls2 != 0 THEN 1 ELSE 0 END +
            CASE WHEN Domain_cls3 != 0 THEN 1 ELSE 0 END +
            CASE WHEN Domain_cls4 != 0 THEN 1 ELSE 0 END) AS nonzero_csl,
           
            CAST((strftime('%s', Datetime) - strftime('%s', LAG(Datetime) OVER (PARTITION BY Device_ID ORDER BY Datetime))) AS REAL) AS diff
            FROM data WHERE Device_ID IN (''' +list_device_str+''')
    ) AS d
    GROUP BY d.Device_ID) as before_normalized
    
    ;
    '''
    cur.execute(sql)
    df = pd.DataFrame(cur.fetchall(), columns = ['Device_ID','total_urls', 'unique_urls','total_dates',# 'min_date', 'max_date',
         'unique_domains', 'unique_cls1', 'diff_mean', 'diff_std', 'diff_max','min_nonzero_cls',
            'max_nonzero_cls','mean_nonzero_cls','std_nonzero_cls','sum_equal_0','sum_equal_1',
               'sum_equal_2', 'sum_equal_3','sum_equal_0_perc','sum_equal_1_perc','sum_equal_2_perc','sum_equal_3_perc' ])
    
    return df

In [6]:
def acf_on_group(x):
    acf_result = acf(x['total_urls'])

    funcs = ['mean','std','min','max','median']

    autocorr = pd.Series(acf_result).agg(funcs)
    autocorr.index = 'acf_'+ autocorr.index

    smoothing_factor = 0.3
    smoothed_ts = x['total_urls'].ewm(alpha=smoothing_factor,adjust=False).mean()
    sm_agg = pd.Series(smoothed_ts).agg(funcs)
    sm_agg.index = 'smoothing_'+ sm_agg.index


    ent =  pd.Series(entropy(x.value_counts(), base=2),index=['enthropy'])

    return pd.Series(pd.concat([autocorr,sm_agg,ent]))


def seasonal_deco(x,period):
    stl_result = STL(x['total_urls'],period = period).fit()
    period = str(period)

    funcs = ['mean','std','min','max',"median"]
    
    trend = stl_result.trend.agg(funcs)
    trend.index = period +'_trend_'+ trend.index

    seasonal = stl_result.seasonal.agg(funcs)
    seasonal.index = period +'_seasonal_'+ seasonal.index

    resid = stl_result.resid.agg(funcs)
    resid.index = period +'_resid_'+ resid.index



    return pd.Series(pd.concat([trend,seasonal,resid]))



In [7]:

def resampled_features(chunk_df):
    cols = [ 'Datetime','Device_ID',
        'URL',
        'Domain_Name',
        'Domain_cls1']
    chunk_df = chunk_df[cols].copy()

    
    #### repeating values
    
    chunk_df['same_Datetime_as_previous'] = chunk_df['Datetime'] == chunk_df['Datetime'].shift(1)
    chunk_df['same_URL_as_previous'] = chunk_df['URL'] == chunk_df['URL'].shift(1)
    chunk_df['same_Domain_Name_as_previous'] = chunk_df['Domain_Name'] == chunk_df['Domain_Name'].shift(1)
    chunk_df['same_Domain_cls1_as_previous'] = (chunk_df['Domain_cls1'] == chunk_df['Domain_cls1'].shift(1))&(chunk_df['Domain_cls1']!=0)
    
    
    repeating_data = chunk_df.groupby('Device_ID',as_index=True).agg(total_urls = ('same_Datetime_as_previous','sum'),
                        repeating_url = ('same_URL_as_previous','sum'),
                        repeating_domain_name = ('same_Domain_Name_as_previous','sum'),
                        repeating_cls1 = ('same_Domain_cls1_as_previous','sum'),
    
    )

    ## resampled data

    chunk_df["Datetime"] = pd.to_datetime(chunk_df["Datetime"])
    
    agg_by_datetime = chunk_df.groupby(['Device_ID','Datetime'],as_index=False).agg(total_urls = ('URL','count'),
                    unique_domains = ('Domain_Name','nunique'),
                    unique_cls = ('Domain_cls1','nunique'),)

    del chunk_df
    
    #resample
    
    daily_level = agg_by_datetime.groupby(['Device_ID',pd.Grouper(freq='D', key='Datetime')])\
                            .agg(['sum','mean','std']).reset_index()
    
    daily_level.columns = [''.join(col) if (col[1] == '') else '_'.join(col) for col in daily_level.columns]
    
    daily_level = daily_level.drop('Datetime',axis=1)
    daily_level = daily_level.groupby('Device_ID').agg(['mean','std'])
    daily_level.columns = ['_daily_'.join(col) for col in daily_level.columns]
    

    ### resample week

    weekly_level = agg_by_datetime.groupby(['Device_ID',pd.Grouper(freq='W', key='Datetime')])\
                            .agg(['sum','mean','std']).reset_index()
    
    weekly_level.columns = [''.join(col) if (col[1] == '') else '_'.join(col) for col in weekly_level.columns]
    
    weekly_level = weekly_level.drop('Datetime',axis=1)
    weekly_level = weekly_level.groupby('Device_ID').agg(['mean','std'])
    weekly_level.columns = ['_weekly_'.join(col) for col in weekly_level.columns]



    resampled_df = pd.concat([daily_level,weekly_level,repeating_data],axis=1)

    
    acf_df = agg_by_datetime.groupby('Device_ID').apply(acf_on_group,include_groups=False)
    seasonal_decomp_7 = agg_by_datetime.groupby('Device_ID').apply(seasonal_deco,period = 7,include_groups=False)
    seasonal_decomp_24 = agg_by_datetime.groupby('Device_ID').apply(seasonal_deco,period = 24,include_groups=False)
    seasonal_decomp_30 = agg_by_datetime.groupby('Device_ID').apply(seasonal_deco,period = 30,include_groups=False)
    
    seasonal_df = pd.concat([acf_df,seasonal_decomp_7,seasonal_decomp_24,seasonal_decomp_30],axis=1)

    
    current_chunk_df = pd.concat([resampled_df,seasonal_df],axis=1).reset_index(drop=True)
    return current_chunk_df
    

In [8]:
from statsmodels.tsa.stattools import acf
from scipy.stats import entropy
from statsmodels.tsa.seasonal import STL

def load_data_in_chunks(device_list, chunk_size=650):
    conn = sqlite3.connect(db)
    
    for i in range(0, len(device_list), chunk_size):
        chunk = device_list[i:i + chunk_size]  # Get the next batch of devices
        list_device_str = ', '.join(map(str, chunk))
        query = f'''SELECT * FROM data WHERE Device_ID IN (''' +list_device_str+''')'''
        df = pd.read_sql_query(query, conn)
        yield df  
    
    conn.close()

In [9]:
def weekly_pivoted(chunk_df):

    chunk_df["Datetime"] = pd.to_datetime(chunk_df["Datetime"])
    
    agg_by_datetime = chunk_df.groupby(['Device_ID','Datetime'],as_index=False).agg(total_urls = ('URL','count'),
                    unique_domains = ('Domain_Name','nunique'),
                    unique_cls = ('Domain_cls1','nunique'),)

    del chunk_df

    funcs = ['mean','std','min','max',"median"]

    week = agg_by_datetime.groupby(['Device_ID',pd.Grouper(freq='W', key='Datetime')])\
                                    .agg(funcs).reset_index()
    
    week.columns = [''.join(col) if (col[1] == '') else '_'.join(col) for col in week.columns]
    week['week_num'] = ((week["Datetime"].dt.day - 1) // 7 + 1)
    week['week_num'] = week['week_num'].map({4:1,5:2,1:3,2:4}).astype(str)
    
    ### fill missing weeks
    
    all_weeks = pd.DataFrame({'week_num': ['1', '2', '3', '4']})
    device_ids = week['Device_ID'].unique()
    full_index = pd.MultiIndex.from_product([device_ids, all_weeks['week_num']], names=['Device_ID', 'week_num'])
    filled_week = week.set_index(['Device_ID', 'week_num']).reindex(full_index).reset_index()
    
    filled_week = filled_week.fillna(0)


    columns_to_pivot = ['total_urls_mean', 'total_urls_std',
           'total_urls_min', 'total_urls_max', 'total_urls_median',
           'unique_domains_mean', 'unique_domains_std', 'unique_domains_min',
           'unique_domains_max', 'unique_domains_median', 'unique_cls_mean',
           'unique_cls_std', 'unique_cls_min', 'unique_cls_max',
           'unique_cls_median',]
    
    
    pivoted_week = filled_week.pivot(index='Device_ID', columns='week_num', values=columns_to_pivot)
    pivoted_week.columns = ['_week_'.join(col) for col in pivoted_week.columns]

    # add diff
    diff_cols = ['Device_ID', 'week_num']
    
    for col in columns_to_pivot:
        filled_week[col + '_diff'] = filled_week.groupby('Device_ID')[col].diff()
        filled_week[col + '_pct_change'] = filled_week.groupby('Device_ID')[col].pct_change()
        diff_cols.append(col + '_diff')
        diff_cols.append(col + '_pct_change')
    
    diff_week_df = filled_week[diff_cols]
    diff_week_df = diff_week_df[diff_week_df['week_num']!='1']


    diff_columns_to_pivot = ['total_urls_mean_diff', 'total_urls_std_diff',
           'total_urls_min_diff', 'total_urls_max_diff', 'total_urls_median_diff',
           'unique_domains_mean_diff', 'unique_domains_std_diff',
           'unique_domains_min_diff', 'unique_domains_max_diff',
           'unique_domains_median_diff', 'unique_cls_mean_diff',
           'unique_cls_std_diff', 'unique_cls_min_diff', 'unique_cls_max_diff',
           'unique_cls_median_diff']
    
    
    pivoted_diff = diff_week_df.pivot(index='Device_ID', columns='week_num', values=diff_columns_to_pivot)
    pivoted_diff.columns = ['_week_'.join(col) for col in pivoted_diff.columns]

    weekly_pivoted_df = pd.concat([pivoted_week,pivoted_diff],axis=1)
    return weekly_pivoted_df
            


In [10]:
sqlite3.sqlite_version

'3.37.2'

In [11]:
#mini = True
mini = False

print('mini :',mini)


if mini:
    db = '/kaggle/input/mafat-mini/mini_training_set.db'
else:
    db = '/kaggle/input/mafat-mini/training_set.db'

conn = sqlite3.connect(db)
    


mini : False


# run program

In [12]:
TRAIN = 1
GET_TARGET = 1 
CRAETE_BASIC_PANEL = 0
CREATE_DAILY_PANEL = 1
CREATE_WEEKLY_PANEL = 1

use_n_devices = False
#use_n_devices = 100


SAVE_PANEL = 1
assert SAVE_PANEL,''' SAVE PANEL IS OFF!!!!'''

In [13]:
%%time 


conn = sqlite3.connect(db)

device_list = pd.read_sql_query("SELECT distinct Device_ID FROM data ", conn)["Device_ID"].tolist()
if use_n_devices :
    device_list = device_list[:use_n_devices]
print(len(device_list))
#conn.close()

6154
CPU times: user 51.5 s, sys: 14 s, total: 1min 5s
Wall time: 3min 47s


In [14]:
if GET_TARGET:
    print('Extract the target variable')
    target_df = target_extract(conn)

    print('Getting indexes for a partition that preserves the proportions of the data')
    s = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=0)
    
    for train_index, test_index in s.split(target_df.Target.values,target_df.Target.values):
        train_target = target_df.iloc[train_index,:]
        test_target = target_df.iloc[test_index,:]
    
        train_device = list(train_target.Device_ID)
        test_device = list(test_target.Device_ID)

    hours_duration = "3" # @param ["2", "3", "4", "6", "8"]
    hours_duration = int(hours_duration)

Extract the target variable
Getting indexes for a partition that preserves the proportions of the data


## prepare panel

### basic panel

In [15]:
%%time 
if CRAETE_BASIC_PANEL:
    print('CRAETE_BASIC_PANEL')
    domain_name_feat_train = relative_domain(conn, train_device)
    domain_name_feat_train = rename_and_16_convert(domain_name_feat_train,'Domain')
    
    cls_name_feat_train = cls_proportion(conn, train_device)
    cls_name_feat_train = rename_and_16_convert(cls_name_feat_train,'cls')
    
    ts_feat_train = avg_relative_entrances_device_id(conn, hours_duration, train_device)
    ts_feat_train = rename_and_16_convert(ts_feat_train,'ts')

    basic_stats_feat_train = basic_stats(conn,train_device)
    
    df_train = pd.merge(domain_name_feat_train, cls_name_feat_train, how ='left', on ='Device_ID')
    df_train = pd.merge(df_train, ts_feat_train, how ='left', on ='Device_ID')
    df_train = pd.merge(df_train, basic_stats_feat_train, how ='left', on ='Device_ID')

    df_train_columns = list(df_train.columns)
    
    
    df_train.to_parquet('df_train', index=False, engine="pyarrow")

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 7.15 µs


### basic panel test

In [16]:
%%time 
if CRAETE_BASIC_PANEL:
    print('CRAETE_BASIC_PANEL_test')
    domain_name_feat_test = relative_domain(conn, test_device)
    domain_name_feat_test = rename_and_16_convert(domain_name_feat_test,'Domain')
    
    cls_name_feat_test = cls_proportion(conn, test_device)
    cls_name_feat_test = rename_and_16_convert(cls_name_feat_test,'cls')
    
    ts_feat_test = avg_relative_entrances_device_id(conn, hours_duration, test_device)
    ts_feat_test = rename_and_16_convert(ts_feat_test,'ts')

    basic_stats_feat_test = basic_stats(conn,test_device)
    
    df_test = pd.merge(domain_name_feat_test, cls_name_feat_test, how ='left', on ='Device_ID')
    df_test = pd.merge(df_test, ts_feat_test, how ='left', on ='Device_ID')
    df_test = pd.merge(df_test, basic_stats_feat_test, how ='left', on ='Device_ID')
    
    df_test = corresponding_columns_training_set(list(df_train.columns), df_test)
    
    df_test.to_parquet('df_test', index=False, engine="pyarrow")

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 8.82 µs


## load panel

In [17]:
if not GET_TARGET and CRAETE_BASIC_PANEL:
    path = '/kaggle/input/panels-and-basic-features-v2/'
   
    df_train = pd.read_parquet(path+'df_train', engine="pyarrow")

    df_test = pd.read_parquet(path+'df_test', engine="pyarrow")

    df_train_columns = list(df_train.columns)

# add features

In [18]:
if GET_TARGET and CRAETE_BASIC_PANEL:
    train_target.to_parquet('train_target', index=False, engine="pyarrow")
    test_target.to_parquet('test_target', index=False, engine="pyarrow")

if not GET_TARGET and CRAETE_BASIC_PANEL:
    train_target = pd.read_parquet('train_target', engine="pyarrow")
    test_target = pd.read_parquet('test_target', engine="pyarrow")

## daily features

In [19]:
%%time
if CREATE_DAILY_PANEL:
    print('CREATE_DAILY_PANEL')
    final_df = pd.DataFrame()
    
    for chunk_df in load_data_in_chunks(device_list, chunk_size=600):
        print('new chunk')
    
        ### agg 
    
        cols = ['Device_ID',
                'Datetime',
                'URL',
                'Domain_Name',
                'Domain_cls1']
        chunk_df = chunk_df[cols].copy()
    
        current_chunk_df = resampled_features(chunk_df)

        del chunk_df
    
        final_df = pd.concat([final_df, current_chunk_df], ignore_index=False)
    
    
    if SAVE_PANEL:
        print(final_df.shape)
        final_df.to_parquet('new_features_df', index=False, engine="pyarrow")


CREATE_DAILY_PANEL
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
(6154, 96)
CPU times: user 2h 51min 34s, sys: 5min 44s, total: 2h 57min 18s
Wall time: 3h 22min 25s


In [20]:
if CREATE_DAILY_PANEL and CRAETE_BASIC_PANEL :
    df_train = pd.merge(df_train, final_df, how ='left', on ='Device_ID')
    
    df_test = pd.merge(df_test, final_df, how ='left', on ='Device_ID')
    
    if SAVE_PANEL:
        print('save added features')
        df_train.to_parquet('df_train_added_features_final', index=False, engine="pyarrow")
        df_test.to_parquet('df_test_added_features_final', index=False, engine="pyarrow")

In [21]:
%%time
if CREATE_WEEKLY_PANEL:
    print('CREATE_WEEKLY_PANEL')
    final_df = pd.DataFrame()
    
    for chunk_df in load_data_in_chunks(device_list, chunk_size=600):
        print('new chunk')
    
        ### agg 
    
        cols = ['Device_ID',
                'Datetime',
                'URL',
                'Domain_Name',
                'Domain_cls1']
        chunk_df = chunk_df[cols].copy()

        current_chunk_df = weekly_pivoted(chunk_df)

        del chunk_df

        final_df = pd.concat([final_df, current_chunk_df], ignore_index=False)
    
    
    if SAVE_PANEL:
        print('final panel')
        print(final_df.shape)
        final_df.to_parquet('new_features_df_set2', index=False, engine="pyarrow")

CREATE_WEEKLY_PANEL
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
new chunk
final panel
(6154, 105)
CPU times: user 40min 33s, sys: 5min 19s, total: 45min 53s
Wall time: 1h 12min 22s


In [22]:
if CREATE_WEEKLY_PANEL and CREATE_DAILY_PANEL and CRAETE_BASIC_PANEL :
    df_train = pd.merge(df_train, final_df, how ='left', on ='Device_ID')
    
    df_test = pd.merge(df_test, final_df, how ='left', on ='Device_ID')

    print(df_train.shape)
    print(df_test.shape)
    
    if SAVE_PANEL:
        print('save added features')
        df_train.to_parquet('df_train_added_features_final', index=False, engine="pyarrow")
        df_test.to_parquet('df_test_added_features_final', index=False, engine="pyarrow")

In [23]:
conn.close()